In [ ]:
import re
import uuid
import time
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from collections import defaultdict

import requests
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, Range, SearchRequest
from fastapi import FastAPI, HTTPException, UploadFile, File
from pydantic import BaseModel
import markdown
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

# Constants
COLLECTION_NAME = "readme_sections"
VECTOR_SIZE = 384  # Size of nomic-embed-text embeddings
OLLAMA_API_URL = "http://localhost:11434/api/embeddings"

@dataclass
class ReadmeSection:
    content: str
    heading: str
    level: int
    parent: Optional[str]
    children: List[str]
    metadata: Dict[str, Any]

class READMEProcessor:
    def __init__(self):
        self.qdrant_client = QdrantClient("localhost", port=6333)
        self._setup_collection()
        self.tfidf_vectorizer = TfidfVectorizer()

    def _setup_collection(self):
        if not self.qdrant_client.get_collection(COLLECTION_NAME):
            self.qdrant_client.create_collection(
                collection_name=COLLECTION_NAME,
                vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE)
            )

    def _get_embedding(self, text: str) -> List[float]:
        response = requests.post(OLLAMA_API_URL, json={
            "model": "nomic-embed-text",
            "prompt": text
        })
        response.raise_for_status()
        return response.json()['embedding']

    def parse_readme(self, content: str) -> List[ReadmeSection]:
        html = markdown.markdown(content)
        soup = BeautifulSoup(html, 'html.parser')
        sections = []
        section_stack = []
        current_section = None

        for elem in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'pre', 'ul', 'ol']):
            if elem.name.startswith('h'):
                level = int(elem.name[1])
                while section_stack and section_stack[-1].level >= level:
                    section_stack.pop()

                parent = section_stack[-1] if section_stack else None
                current_section = ReadmeSection(
                    content=elem.text,
                    heading=elem.text,
                    level=level,
                    parent=parent.heading if parent else None,
                    children=[],
                    metadata={}
                )
                if parent:
                    parent.children.append(current_section.heading)
                sections.append(current_section)
                section_stack.append(current_section)
            else:
                if current_section:
                    current_section.content += "\n" + elem.text

        return sections

    def process_readme(self, content: str):
        sections = self.parse_readme(content)
        section_graph = self._build_section_graph(sections)
        for section in sections:
            self._add_section_to_qdrant(section, section_graph)

    def _build_section_graph(self, sections: List[ReadmeSection]) -> nx.DiGraph:
        G = nx.DiGraph()
        for section in sections:
            G.add_node(section.heading, level=section.level)
            if section.parent:
                G.add_edge(section.parent, section.heading)
        return G

    def _add_section_to_qdrant(self, section: ReadmeSection, section_graph: nx.DiGraph):
        vector = self._get_embedding(section.content)
        point_id = str(uuid.uuid4())
        timestamp = time.time()

        # Calculate centrality and other graph-based features
        centrality = nx.degree_centrality(section_graph)[section.heading]
        depth = nx.shortest_path_length(section_graph, source=list(section_graph.nodes)[0], target=section.heading)

        payload = {
            "content": section.content,
            "heading": section.heading,
            "level": section.level,
            "parent": section.parent,
            "children": section.children,
            "metadata": {
                **section.metadata,
                "timestamp": timestamp,
                "centrality": centrality,
                "depth": depth,
                "access_count": 0,
                "relevance_score": 1.0
            }
        }

        self.qdrant_client.upsert(
            collection_name=COLLECTION_NAME,
            points=[PointStruct(id=point_id, vector=vector, payload=payload)]
        )

    def search_sections(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        query_vector = self._get_embedding(query)

        # Perform semantic search
        search_result = self.qdrant_client.search(
            collection_name=COLLECTION_NAME,
            query_vector=query_vector,
            limit=top_k * 2  # Retrieve more results for re-ranking
        )

        # Extract contents for TF-IDF re-ranking
        contents = [hit.payload['content'] for hit in search_result]
        tfidf_matrix = self.tfidf_vectorizer.fit_transform([query] + contents)
        
        # Calculate TF-IDF similarities
        tfidf_similarities = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:])[0]
        
        # Combine semantic and TF-IDF scores
        combined_scores = [(hit, 0.7 * hit.score + 0.3 * tfidf_sim) 
                           for hit, tfidf_sim in zip(search_result, tfidf_similarities)]
        
        # Sort by combined score and take top_k
        combined_scores.sort(key=lambda x: x[1], reverse=True)
        top_results = combined_scores[:top_k]

        results = []
        for hit, score in top_results:
            section = hit.payload
            section['score'] = score
            self._update_section_relevance(hit.id, score)
            results.append(section)

        return results

    def _update_section_relevance(self, point_id: str, score: float):
        current_payload = self.qdrant_client.retrieve(COLLECTION_NAME, [point_id])[0].payload
        current_payload['metadata']['access_count'] += 1
        current_payload['metadata']['relevance_score'] = (current_payload['metadata']['relevance_score'] + score) / 2

        self.qdrant_client.upsert(
            collection_name=COLLECTION_NAME,
            points=[PointStruct(id=point_id, payload=current_payload)]
        )

    def get_context(self, section_heading: str, depth: int = 1) -> Dict[str, Any]:
        filter_condition = Filter(
            must=[FieldCondition(key="heading", match={'value': section_heading})]
        )
        results = self.qdrant_client.scroll(
            collection_name=COLLECTION_NAME,
            scroll_filter=filter_condition,
            limit=1
        )
        if not results.points:
            return {}

        section = results.points[0].payload
        context = {
            "current": section,
            "parent": None,
            "children": [],
            "siblings": []
        }

        if section['parent']:
            parent_filter = Filter(
                must=[FieldCondition(key="heading", match={'value': section['parent']})]
            )
            parent_results = self.qdrant_client.scroll(
                collection_name=COLLECTION_NAME,
                scroll_filter=parent_filter,
                limit=1
            )
            if parent_results.points:
                context["parent"] = parent_results.points[0].payload

        if depth > 0:
            for child_heading in section['children']:
                child_context = self.get_context(child_heading, depth - 1)
                if child_context:
                    context["children"].append(child_context["current"])

            if context["parent"]:
                for sibling_heading in context["parent"]["children"]:
                    if sibling_heading != section_heading:
                        sibling_context = self.get_context(sibling_heading, 0)
                        if sibling_context:
                            context["siblings"].append(sibling_context["current"])

        return context

    def prune_sections(self, threshold: float = 0.5, max_age_days: int = 30):
        current_time = time.time()
        max_age_seconds = max_age_days * 24 * 60 * 60

        filter_condition = Filter(
            must=[
                FieldCondition(
                    key="metadata.relevance_score",
                    range=Range(lt=threshold)
                ),
                FieldCondition(
                    key="metadata.timestamp",
                    range=Range(lt=current_time - max_age_seconds)
                )
            ]
        )

        self.qdrant_client.delete(
            collection_name=COLLECTION_NAME,
            points_selector=filter_condition
        )

# FastAPI app
app = FastAPI()
readme_processor = READMEProcessor()

@app.post("/process_readme")
async def process_readme(file: UploadFile = File(...)):
    content = await file.read()
    readme_processor.process_readme(content.decode())
    return {"message": "README processed successfully"}

@app.post("/search")
async def search(query: str, top_k: int = 5):
    results = readme_processor.search_sections(query, top_k)
    return {"results": results}

@app.get("/context/{section_heading}")
async def get_context(section_heading: str, depth: int = 1):
    context = readme_processor.get_context(section_heading, depth)
    return {"context": context}

@app.post("/prune")
async def prune(threshold: float = 0.5, max_age_days: int = 30):
    readme_processor.prune_sections(threshold, max_age_days)
    return {"message": "Pruning completed successfully"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)

In [4]:
import os
import uuid
import time
import logging
from typing import List, Dict, Any, Optional
from dataclasses import dataclass

import requests
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, Range
from fastapi import FastAPI, HTTPException, UploadFile, File
from pydantic import BaseModel
import markdown
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from xgboost import XGBRanker
import networkx as nx

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Qdrant client
qdrant_client = QdrantClient(host="localhost", port=6333)

# Constants
COLLECTION_NAME = "advanced_readme_sections"
VECTOR_SIZE = 384

# Create collection if it doesn't exist
try:
    qdrant_client.get_collection(COLLECTION_NAME)
    logger.info(f"Collection '{COLLECTION_NAME}' already exists.")
except Exception:
    logger.info(f"Creating collection '{COLLECTION_NAME}'.")
    qdrant_client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.EUCLID)
    )

# Initialize FastAPI app
app = FastAPI()

@dataclass
class ReadmeSection:
    content: str
    heading: str
    level: int
    parent: Optional[str]
    children: List[str]
    metadata: Dict[str, Any]
    vector: List[float] = None

def get_embedding(text: str) -> np.ndarray:
    OLLAMA_API_URL = "http://localhost:11434/api/embeddings"
    response = requests.post(OLLAMA_API_URL, json={
        "model": "nomic-embed-text",
        "prompt": text
    })
    response.raise_for_status()
    return np.array(response.json()['embedding'])

def parse_readme(content: str) -> List[ReadmeSection]:
    html = markdown.markdown(content)
    soup = BeautifulSoup(html, 'html.parser')
    sections = []
    section_stack = []
    current_section = None

    for elem in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'pre', 'ul', 'ol']):
        if elem.name.startswith('h'):
            level = int(elem.name[1])
            while section_stack and section_stack[-1].level >= level:
                section_stack.pop()

            parent = section_stack[-1] if section_stack else None
            current_section = ReadmeSection(
                content='',
                heading=elem.text,
                level=level,
                parent=parent.heading if parent else None,
                children=[],
                metadata={}
            )
            if parent:
                parent.children.append(current_section.heading)
            sections.append(current_section)
            section_stack.append(current_section)
        else:
            if current_section:
                current_section.content += "\n" + elem.text

    return sections

def build_section_graph(sections: List[ReadmeSection]) -> nx.DiGraph:
    G = nx.DiGraph()
    for section in sections:
        G.add_node(section.heading, level=section.level)
        if section.parent:
            G.add_edge(section.parent, section.heading)
    return G

def cluster_sections(sections: List[ReadmeSection], n_clusters: int = 10):
    embeddings = np.array([section.vector for section in sections])
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings)
    for section, label in zip(sections, cluster_labels):
        section.metadata['cluster'] = int(label)

def add_section_to_qdrant(section: ReadmeSection, section_graph: nx.DiGraph):
    try:
        vector = get_embedding(section.content)
    except Exception as e:
        logger.error(f"Failed to get embedding for section '{section.heading}': {e}")
        return
    
    point_id = str(uuid.uuid4())
    timestamp = time.time()

    centrality = nx.degree_centrality(section_graph).get(section.heading, 0)
    try:
        depth = nx.shortest_path_length(section_graph, source=list(section_graph.nodes)[0], target=section.heading)
    except nx.NetworkXNoPath:
        depth = 0

    payload = {
        "content": section.content,
        "heading": section.heading,
        "level": section.level,
        "parent": section.parent,
        "children": section.children,
        "metadata": {
            **section.metadata,
            "timestamp": timestamp,
            "centrality": centrality,
            "depth": depth,
            "access_count": 0,
            "relevance_score": 1.0
        }
    }

    qdrant_client.upsert(
        collection_name=COLLECTION_NAME,
        points=[PointStruct(id=point_id, vector=vector.tolist(), payload=payload)]
    )
    logger.info(f"Section '{section.heading}' added to Qdrant with ID {point_id}.")

knn_model: Optional[NearestNeighbors] = None
point_id_mapping: Dict[int, str] = {}

def build_knn_index():
    global knn_model, point_id_mapping
    logger.info("Building KNN index...")
    all_points = qdrant_client.scroll(collection_name=COLLECTION_NAME, limit=10000)
    
    if not all_points or not all_points[0]:
        logger.warning("No points found in the collection. KNN index not built.")
        knn_model = None
        point_id_mapping = {}
        return
    
    embeddings = np.array([point.vector for point in all_points[0]])
    
    if embeddings.size == 0:
        logger.warning("Embeddings array is empty. KNN index not built.")
        knn_model = None
        point_id_mapping = {}
        return
    
    knn_model = NearestNeighbors(n_neighbors=10, algorithm='auto', metric='euclidean')
    knn_model.fit(embeddings)
    point_id_mapping = {i: point.id for i, point in enumerate(all_points[0])}
    logger.info(f"KNN index built successfully with {len(point_id_mapping)} points.")

tfidf_vectorizer = TfidfVectorizer()

def calculate_tfidf_similarity(query: str, document: str) -> float:
    tfidf_matrix = tfidf_vectorizer.fit_transform([query, document])
    return (tfidf_matrix * tfidf_matrix.T).A[0, 1]

def prepare_training_data(query: str, sections: List[ReadmeSection]):
    features = []
    labels = []
    for section in sections:
        feature_vector = [
            section.metadata.get('tfidf_similarity', 0.0),
            section.metadata.get('semantic_similarity', 0.0),
            section.metadata.get('centrality', 0.0),
            section.level,
            section.metadata.get('cluster', 0)
        ]
        features.append(feature_vector)
        labels.append(section.metadata.get('relevance_label', 1))  # Placeholder
    return np.array(features), np.array(labels)

xgb_ranker = XGBRanker(
    objective='rank:pairwise',
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100
)

def search_sections(query: str, top_k: int = 5) -> List[Dict[str, Any]]:
    if knn_model is None:
        logger.warning("KNN model is not built. No search can be performed.")
        return []
    
    query_vector = get_embedding(query).reshape(1, -1)
    distances, indices = knn_model.kneighbors(query_vector)
    nearest_points = [point_id_mapping[idx] for idx in indices[0]]
    
    sections = []
    for point_id in nearest_points:
        point = qdrant_client.retrieve(collection_name=COLLECTION_NAME, ids=[point_id])[0]
        section = point.payload
        section['vector'] = point.vector
        tfidf_sim = calculate_tfidf_similarity(query, section['content'])
        section['metadata']['tfidf_similarity'] = tfidf_sim
        semantic_sim = 1 / (1 + distances[0][indices[0].tolist().index(point_id_mapping.index(point_id))])
        section['metadata']['semantic_similarity'] = semantic_sim
        sections.append(section)
    
    if not sections:
        return []
    
    X_test, _ = prepare_training_data(query, sections)
    relevance_scores = xgb_ranker.predict(X_test)
    
    for section, score in zip(sections, relevance_scores):
        section['score'] = score
    sections.sort(key=lambda x: x['score'], reverse=True)
    
    for section in sections[:top_k]:
        update_section_relevance(section['id'], section['score'])
    return sections[:top_k]

def update_section_relevance(point_id: str, score: float):
    current_payload = qdrant_client.retrieve(
        collection_name=COLLECTION_NAME, ids=[point_id]
    )[0].payload
    current_payload['metadata']['access_count'] += 1
    current_payload['metadata']['relevance_score'] = (
        current_payload['metadata']['relevance_score'] + score
    ) / 2

    qdrant_client.upsert(
        collection_name=COLLECTION_NAME,
        points=[PointStruct(id=point_id, payload=current_payload)]
    )
    logger.info(f"Updated relevance for point ID {point_id}.")

def get_context(section_heading: str, depth: int = 1) -> Dict[str, Any]:
    filter_condition = Filter(
        must=[FieldCondition(key="heading", match={'value': section_heading})]
    )
    results = qdrant_client.scroll(
        collection_name=COLLECTION_NAME,
        scroll_filter=filter_condition,
        limit=1
    )
    if not results.points:
        return {}

    section = results.points[0].payload
    context = {
        "current": section,
        "parent": None,
        "children": [],
        "siblings": []
    }

    if section['parent']:
        parent_filter = Filter(
            must=[FieldCondition(key="heading", match={'value': section['parent']})]
        )
        parent_results = qdrant_client.scroll(
            collection_name=COLLECTION_NAME,
            scroll_filter=parent_filter,
            limit=1
        )
        if parent_results.points:
            context["parent"] = parent_results.points[0].payload

    if depth > 0 and 'children' in section:
        for child_heading in section['children']:
            child_context = get_context(child_heading, depth - 1)
            if child_context:
                context["children"].append(child_context["current"])

    if context["parent"] and 'children' in context["parent"]:
        for sibling_heading in context["parent"]["children"]:
            if sibling_heading != section_heading:
                sibling_context = get_context(sibling_heading, 0)
                if sibling_context:
                    context["siblings"].append(sibling_context["current"])

    return context

def prune_sections(threshold: float = 0.5, max_age_days: int = 30):
    current_time = time.time()
    max_age_seconds = max_age_days * 24 * 60 * 60

    filter_condition = Filter(
        must=[
            FieldCondition(
                key="metadata.relevance_score",
                range=Range(lt=threshold)
            ),
            FieldCondition(
                key="metadata.timestamp",
                range=Range(lt=current_time - max_age_seconds)
            )
        ]
    )

    qdrant_client.delete(
        collection_name=COLLECTION_NAME,
        points_selector=filter_condition
    )
    logger.info("Pruned low-relevance and old sections.")

@app.post("/process_readme")
async def process_readme_api(file: UploadFile = File(...)):
    content = await file.read()
    sections = parse_readme(content.decode())
    section_graph = build_section_graph(sections)
    for section in sections:
        section.vector = get_embedding(section.content).tolist()
    cluster_sections(sections)
    for section in sections:
        add_section_to_qdrant(section, section_graph)
    build_knn_index()
    return {"message": "README processed successfully"}

@app.post("/search")
async def search_api(query: str, top_k: int = 5):
    results = search_sections(query, top_k)
    return {"results": results}

@app.get("/context/{section_heading}")
async def get_context_api(section_heading: str, depth: int = 1):
    context = get_context(section_heading, depth)
    return {"context": context}

@app.post("/prune")
async def prune_api(threshold: float = 0.5, max_age_days: int = 30):
    prune_sections(threshold, max_age_days)
    return {"message": "Pruning completed successfully"}

@app.post("/rebuild_knn_index")
async def rebuild_knn_index():
    build_knn_index()
    return {"message": "KNN index rebuilt successfully"}

# if __name__ == "__main__":
#     import uvicorn
#     build_knn_index()  # This will now handle empty collections gracefully
#     uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:httpx:HTTP Request: GET http://localhost:6333/collections/advanced_readme_sections "HTTP/1.1 200 OK"
INFO:__main__:Collection 'advanced_readme_sections' already exists.


In [6]:
!pip install xgboost


  Using cached xgboost-2.1.1-py3-none-win_amd64.whl.metadata (2.1 kB)
Using cached xgboost-2.1.1-py3-none-win_amd64.whl (124.9 MB)


In [14]:
# Install Required Dependencies
!pip install fastapi uvicorn requests numpy qdrant-client markdown beautifulsoup4 scikit-learn xgboost networkx nest_asyncio python-dotenv sentence-transformers

# Comprehensive Implementation in One Code Block

import os
import uuid
import time
import math
import logging
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import asyncio
import requests
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct, Filter, FieldCondition, Range
from fastapi import FastAPI, HTTPException, UploadFile, File
from pydantic import BaseModel
import markdown
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from xgboost import XGBRanker
import networkx as nx
import nest_asyncio
import uvicorn
from threading import Thread
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Qdrant clients for both collections
qdrant_client_readme = QdrantClient(host="localhost", port=6333)
qdrant_client_mind = QdrantClient(host="localhost", port=6333)

# Constants for Readme Sections
COLLECTION_NAME_README = "advanced_readme_sections"
VECTOR_SIZE_README = 384  # Adjust based on your embedding model

# Constants for Memory Manager
COLLECTION_NAME_MIND = "Mind"
VECTOR_SIZE_MIND = 384  # Example size; adjust based on SentenceTransformer model

# Create Readme Sections Collection if it doesn't exist
try:
    qdrant_client_readme.get_collection(COLLECTION_NAME_README)
    logger.info(f"Collection '{COLLECTION_NAME_README}' already exists.")
except Exception:
    logger.info(f"Creating collection '{COLLECTION_NAME_README}'.")
    qdrant_client_readme.create_collection(
        collection_name=COLLECTION_NAME_README,
        vectors_config=VectorParams(size=VECTOR_SIZE_README, distance=Distance.EUCLID)
    )

# Create Mind Collection if it doesn't exist
try:
    qdrant_client_mind.get_collection(COLLECTION_NAME_MIND)
    logger.info(f"Collection '{COLLECTION_NAME_MIND}' already exists.")
except Exception:
    logger.info(f"Creating collection '{COLLECTION_NAME_MIND}'.")
    # Initialize SentenceTransformer for MemoryManager
    memory_model = SentenceTransformer('all-MiniLM-L6-v2')
    VECTOR_SIZE_MIND = memory_model.get_sentence_embedding_dimension()
    qdrant_client_mind.create_collection(
        collection_name=COLLECTION_NAME_MIND,
        vectors_config=VectorParams(size=VECTOR_SIZE_MIND, distance=Distance.COSINE)
    )

# Define Data Models
@dataclass
class ReadmeSection:
    content: str
    heading: str
    level: int
    parent: Optional[str]
    children: List[str]
    metadata: Dict[str, Any]
    vector: Optional[List[float]] = None

class MemoryPacket(BaseModel):
    vector: List[float]
    content: str
    metadata: Dict[str, Any]

# Define MemoryManager Class
class MemoryManager:
    def __init__(self, qdrant_client: QdrantClient, collection_name: str, model_name: str = 'all-MiniLM-L6-v2'):
        self.qdrant_client = qdrant_client
        self.collection_name = collection_name
        self.model = SentenceTransformer(model_name)
        self._setup_collection()

    def _setup_collection(self):
        try:
            self.qdrant_client.get_collection(self.collection_name)
            logger.info(f"Collection '{self.collection_name}' exists.")
        except Exception:
            logger.info(f"Creating collection '{self.collection_name}'.")
            self.qdrant_client.create_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(size=self.model.get_sentence_embedding_dimension(), distance=Distance.COSINE)
            )

    async def create_memory(self, content: str, metadata: Dict[str, Any]):
        vector = self.model.encode(content).tolist()
        memory_packet = MemoryPacket(vector=vector, content=content, metadata=metadata)
        point_id = str(uuid.uuid4())

        try:
            self.qdrant_client.upsert(
                collection_name=self.collection_name,
                points=[PointStruct(id=point_id, vector=vector, payload=memory_packet.dict())]
            )
            logger.info(f"Memory created successfully with ID: {point_id}")
        except Exception as e:
            logger.error(f"Error creating memory: {e}")

    async def recall_memory(self, query_content: str, top_k: int = 5):
        query_vector = self.model.encode(query_content).tolist()

        try:
            results = self.qdrant_client.search(
                collection_name=self.collection_name,
                query_vector=query_vector,
                limit=top_k
            )

            memories = [MemoryPacket(**hit.payload) for hit in results]

            for memory in memories:
                self._update_relevance(memory, query_vector)

            ranked_memories = sorted(
                memories,
                key=lambda mem: (
                    mem.metadata['semantic_relativity'] * mem.metadata['memetic_similarity'] * mem.metadata['gravitational_pull']
                ),
                reverse=True
            )

            return [{
                "content": memory.content,
                "metadata": memory.metadata
            } for memory in ranked_memories[:top_k]]
        except Exception as e:
            logger.error(f"Error recalling memory: {e}")
            return []

    def _update_relevance(self, memory: MemoryPacket, query_vector: List[float]):
        memory.metadata["semantic_relativity"] = self._calculate_cosine_similarity(memory.vector, query_vector)
        memory.metadata["memetic_similarity"] = self._calculate_memetic_similarity(memory.metadata)
        memory.metadata["gravitational_pull"] = self._calculate_gravitational_pull(memory)
        memory.metadata["spacetime_coordinate"] = self._calculate_spacetime_coordinate(memory)

    @staticmethod
    def _calculate_cosine_similarity(vector_a: List[float], vector_b: List[float]) -> float:
        dot_product = sum(a * b for a, b in zip(vector_a, vector_b))
        magnitude_a = math.sqrt(sum(a ** 2 for a in vector_a))
        magnitude_b = math.sqrt(sum(b ** 2 for b in vector_b))

        if magnitude_a == 0 or magnitude_b == 0:
            return 0.0

        return dot_product / (magnitude_a * magnitude_b)

    @staticmethod
    def _calculate_memetic_similarity(metadata: Dict[str, Any]) -> float:
        tags = set(metadata.get("tags", []))
        reference_tags = set(metadata.get("reference_tags", []))

        if not tags or not reference_tags:
            return 1.0

        intersection = len(tags.intersection(reference_tags))
        union = len(tags.union(reference_tags))

        return intersection / union if union > 0 else 1.0

    @staticmethod
    def _calculate_gravitational_pull(memory: MemoryPacket) -> float:
        vector_magnitude = math.sqrt(sum(x ** 2 for x in memory.vector))
        recall_count = memory.metadata.get("recall_count", 0)
        memetic_similarity = memory.metadata.get("memetic_similarity", 1.0)
        semantic_relativity = memory.metadata.get("semantic_relativity", 1.0)

        return vector_magnitude * (1 + math.log1p(recall_count)) * memetic_similarity * semantic_relativity

    @staticmethod
    def _calculate_spacetime_coordinate(memory: MemoryPacket) -> float:
        time_decay_factor = 1 + (time.time() - memory.metadata.get("timestamp", time.time()))
        return memory.metadata["gravitational_pull"] / time_decay_factor

    async def prune_memories(self, threshold: float = 1e-5, max_age_days: int = 30):
        try:
            current_time = time.time()
            max_age_seconds = max_age_days * 24 * 60 * 60

            filter_condition = Filter(
                must=[
                    FieldCondition(
                        key="metadata.relevance_score",
                        range=Range(lt=threshold)
                    ),
                    FieldCondition(
                        key="metadata.timestamp",
                        range=Range(lt=current_time - max_age_seconds)
                    )
                ]
            )

            self.qdrant_client.delete(
                collection_name=self.collection_name,
                filter=filter_condition
            )
            logger.info("Pruned low-relevance and old memories.")
        except Exception as e:
            logger.error(f"Error pruning memories: {e}")

    async def purge_all_memories(self):
        try:
            self.qdrant_client.delete_collection(self.collection_name)
            self._setup_collection()
            logger.info(f"Purged all memories in the collection '{self.collection_name}'.")
        except Exception as e:
            logger.error(f"Error purging all memories: {e}")
            raise e

    async def recall_memory_with_metadata(self, query_content: str, search_metadata: Dict[str, Any], top_k: int = 10):
        try:
            query_vector = self.model.encode(query_content).tolist()
            results = self.qdrant_client.search(
                collection_name=self.collection_name,
                query_vector=query_vector,
                limit=top_k
            )

            memories = [MemoryPacket(**hit.payload) for hit in results]

            matching_memories = []
            for memory in memories:
                memory_metadata = memory.metadata
                if all(memory_metadata.get(key) == value for key, value in search_metadata.items()):
                    matching_memories.append({
                        "content": memory.content,
                        "metadata": memory_metadata
                    })

            if not matching_memories:
                return {"message": "No matching memories found"}

            return {"memories": matching_memories}
        except Exception as e:
            logger.error(f"Error recalling memories by metadata: {str(e)}")
            return {"message": "Error during memory recall"}

    async def delete_memories_by_metadata(self, metadata: Dict[str, Any]):
        try:
            # Scroll through all memories in the collection
            scroll_result = self.qdrant_client.scroll(self.collection_name, limit=1000)

            memories_to_delete = []
            for point in scroll_result:
                point_metadata = point.payload.get("metadata", {})
                if all(point_metadata.get(key) == value for key, value in metadata.items()):
                    memories_to_delete.append(point.id)

            if memories_to_delete:
                self.qdrant_client.delete(
                    collection_name=self.collection_name,
                    points_selector={"points": memories_to_delete}
                )
                logger.info(f"Deleted {len(memories_to_delete)} memories matching the metadata.")
            else:
                logger.info("No memories found matching the specified metadata.")
        except Exception as e:
            logger.error(f"Error deleting memories by metadata: {str(e)}")

# Initialize MemoryManager for Mind Collection
memory_manager = MemoryManager(
    qdrant_client=qdrant_client_mind,
    collection_name=COLLECTION_NAME_MIND,
    model_name='all-MiniLM-L6-v2'
)

# Utility Functions for Readme Processing

def get_embedding(text: str) -> np.ndarray:
    OLLAMA_API_URL = os.getenv("OLLAMA_API_URL", "http://localhost:11434/api/embeddings")
    try:
        response = requests.post(OLLAMA_API_URL, json={
            "model": "nomic-embed-text",
            "prompt": text
        })
        response.raise_for_status()
        return np.array(response.json()['embedding'])
    except Exception as e:
        logger.error(f"Error fetching embedding: {e}")
        raise

def parse_readme(content: str) -> List[ReadmeSection]:
    html = markdown.markdown(content)
    soup = BeautifulSoup(html, 'html.parser')
    sections = []
    section_stack = []
    current_section = None

    for elem in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'pre', 'ul', 'ol']):
        if elem.name.startswith('h'):
            level = int(elem.name[1])
            while section_stack and section_stack[-1].level >= level:
                section_stack.pop()

            parent = section_stack[-1] if section_stack else None
            current_section = ReadmeSection(
                content='',
                heading=elem.text.strip(),
                level=level,
                parent=parent.heading if parent else None,
                children=[],
                metadata={}
            )
            if parent:
                parent.children.append(current_section.heading)
            sections.append(current_section)
            section_stack.append(current_section)
        else:
            if current_section:
                current_section.content += "\n" + elem.get_text(separator=" ", strip=True)

    return sections

def build_section_graph(sections: List[ReadmeSection]) -> nx.DiGraph:
    G = nx.DiGraph()
    for section in sections:
        G.add_node(section.heading, level=section.level)
        if section.parent:
            G.add_edge(section.parent, section.heading)
    return G

def cluster_sections(sections: List[ReadmeSection], n_clusters: int = 10):
    embeddings = np.array([section.vector for section in sections if section.vector is not None])
    if embeddings.size == 0:
        logger.warning("No embeddings available for clustering.")
        return

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings)
    for section, label in zip([s for s in sections if s.vector is not None], cluster_labels):
        section.metadata['cluster'] = int(label)

def add_section_to_qdrant(section: ReadmeSection, section_graph: nx.DiGraph):
    if not section.vector:
        logger.error(f"Section '{section.heading}' has no vector.")
        return

    point_id = str(uuid.uuid4())
    timestamp = time.time()

    centrality = nx.degree_centrality(section_graph).get(section.heading, 0)
    try:
        depth = nx.shortest_path_length(section_graph, source=list(section_graph.nodes)[0], target=section.heading)
    except nx.NetworkXNoPath:
        depth = 0

    payload = {
        "content": section.content,
        "heading": section.heading,
        "level": section.level,
        "parent": section.parent,
        "children": section.children,
        "metadata": {
            **section.metadata,
            "timestamp": timestamp,
            "centrality": centrality,
            "depth": depth,
            "access_count": 0,
            "relevance_score": 1.0
        }
    }

    try:
        qdrant_client_readme.upsert(
            collection_name=COLLECTION_NAME_README,
            points=[PointStruct(id=point_id, vector=section.vector, payload=payload)]
        )
        logger.info(f"Section '{section.heading}' added to Qdrant with ID {point_id}.")
    except Exception as e:
        logger.error(f"Failed to upsert section '{section.heading}': {e}")

knn_model_readme: Optional[NearestNeighbors] = None
point_id_mapping_readme: Dict[int, str] = {}

def build_knn_index_readme():
    global knn_model_readme, point_id_mapping_readme
    logger.info("Building KNN index for Readme Sections...")
    try:
        # Scroll retrieves points in batches; adjust batch size as needed
        all_points = []
        scroll_response = qdrant_client_readme.scroll(collection_name=COLLECTION_NAME_README, limit=10000)
        while scroll_response:
            all_points.extend(scroll_response.points)
            if scroll_response.next_page_offset:
                scroll_response = qdrant_client_readme.scroll(collection_name=COLLECTION_NAME_README, limit=10000, offset=scroll_response.next_page_offset)
            else:
                break

        if not all_points:
            logger.warning("No points found in the Readme collection. KNN index not built.")
            knn_model_readme = None
            point_id_mapping_readme = {}
            return

        embeddings = np.array([point.vector for point in all_points])
        if embeddings.size == 0:
            logger.warning("Embeddings array is empty for Readme sections. KNN index not built.")
            knn_model_readme = None
            point_id_mapping_readme = {}
            return

        knn_model_readme = NearestNeighbors(n_neighbors=10, algorithm='auto', metric='euclidean')
        knn_model_readme.fit(embeddings)
        point_id_mapping_readme = {i: point.id for i, point in enumerate(all_points)}
        logger.info(f"KNN index for Readme sections built successfully with {len(point_id_mapping_readme)} points.")
    except Exception as e:
        logger.error(f"Error building KNN index for Readme sections: {e}")
        knn_model_readme = None
        point_id_mapping_readme = {}

tfidf_vectorizer = TfidfVectorizer()

def calculate_tfidf_similarity(query: str, document: str) -> float:
    tfidf_matrix = tfidf_vectorizer.fit_transform([query, document])
    return (tfidf_matrix * tfidf_matrix.T).A[0, 1]

def prepare_training_data(query: str, sections: List[Dict[str, Any]]):
    features = []
    labels = []
    for section in sections:
        feature_vector = [
            section['metadata'].get('tfidf_similarity', 0.0),
            section['metadata'].get('semantic_similarity', 0.0),
            section['metadata'].get('centrality', 0.0),
            section['level'],
            section['metadata'].get('cluster', 0)
        ]
        features.append(feature_vector)
        labels.append(section['metadata'].get('relevance_label', 1))  # Placeholder
    return np.array(features), np.array(labels)

xgb_ranker = XGBRanker(
    objective='rank:pairwise',
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100
)

def train_xgb_ranker():
    try:
        # Placeholder: Implement actual training logic
        # This should be done offline with proper labeled data
        # For demonstration, we'll skip training
        logger.info("Training XGBRanker is not implemented. Using default model.")
    except Exception as e:
        logger.error(f"Error training XGBRanker: {e}")

# Train the ranker (currently a placeholder)
train_xgb_ranker()

def search_sections(query: str, top_k: int = 5) -> List[Dict[str, Any]]:
    if knn_model_readme is None:
        logger.warning("KNN model for Readme sections is not built. No search can be performed.")
        return []

    try:
        query_vector = get_embedding(query).reshape(1, -1)
    except Exception as e:
        logger.error(f"Failed to get embedding for query '{query}': {e}")
        return []

    try:
        distances, indices = knn_model_readme.kneighbors(query_vector)
    except Exception as e:
        logger.error(f"Error during KNN search: {e}")
        return []

    nearest_points = [point_id_mapping_readme[idx] for idx in indices[0]]

    sections = []
    for idx, point_id in enumerate(nearest_points):
        try:
            points = qdrant_client_readme.retrieve(collection_name=COLLECTION_NAME_README, ids=[point_id])
            if not points:
                continue
            point = points[0]
            section = point.payload
            section['vector'] = point.vector.tolist()
            tfidf_sim = calculate_tfidf_similarity(query, section['content'])
            section['metadata']['tfidf_similarity'] = tfidf_sim
            # Use the distance directly
            semantic_sim = 1 / (1 + distances[0][idx])
            section['metadata']['semantic_similarity'] = semantic_sim
            sections.append(section)
        except Exception as e:
            logger.error(f"Error retrieving section '{point_id}': {e}")

    if not sections:
        return []

    X_test, _ = prepare_training_data(query, sections)
    if X_test.size == 0:
        logger.warning("No features available for ranking.")
        return []

    try:
        relevance_scores = xgb_ranker.predict(X_test)
    except Exception as e:
        logger.error(f"Error during ranking: {e}")
        relevance_scores = np.ones(len(sections))  # Fallback

    for section, score in zip(sections, relevance_scores):
        section['score'] = score
    sections.sort(key=lambda x: x['score'], reverse=True)

    for section in sections[:top_k]:
        update_section_relevance(section['id'], section['score'])

    return sections[:top_k]

def update_section_relevance(point_id: str, score: float):
    try:
        points = qdrant_client_readme.retrieve(collection_name=COLLECTION_NAME_README, ids=[point_id])
        if not points:
            logger.warning(f"Point ID '{point_id}' not found for relevance update.")
            return
        current_payload = points[0].payload
        current_payload['metadata']['access_count'] += 1
        current_payload['metadata']['relevance_score'] = (
            current_payload['metadata']['relevance_score'] + score
        ) / 2

        qdrant_client_readme.upsert(
            collection_name=COLLECTION_NAME_README,
            points=[PointStruct(id=point_id, vector=points[0].vector.tolist(), payload=current_payload)]
        )
        logger.info(f"Updated relevance for point ID {point_id}.")
    except Exception as e:
        logger.error(f"Error updating relevance for point ID '{point_id}': {e}")

def get_context(section_heading: str, depth: int = 1) -> Dict[str, Any]:
    try:
        filter_condition = Filter(
            must=[FieldCondition(key="heading", match={'value': section_heading})]
        )
        results = qdrant_client_readme.scroll(
            collection_name=COLLECTION_NAME_README,
            filter=filter_condition,
            limit=1
        )
        if not results.points:
            return {}

        section = results.points[0].payload
        context = {
            "current": section,
            "parent": None,
            "children": [],
            "siblings": []
        }

        if section.get('parent'):
            parent_filter = Filter(
                must=[FieldCondition(key="heading", match={'value': section['parent']})]
            )
            parent_results = qdrant_client_readme.scroll(
                collection_name=COLLECTION_NAME_README,
                filter=parent_filter,
                limit=1
            )
            if parent_results.points:
                context["parent"] = parent_results.points[0].payload

        if depth > 0 and 'children' in section:
            for child_heading in section['children']:
                child_context = get_context(child_heading, depth - 1)
                if child_context:
                    context["children"].append(child_context["current"])

        if context.get("parent") and 'children' in context["parent"]:
            for sibling_heading in context["parent"]["children"]:
                if sibling_heading != section_heading:
                    sibling_context = get_context(sibling_heading, 0)
                    if sibling_context:
                        context["siblings"].append(sibling_context["current"])

        return context
    except Exception as e:
        logger.error(f"Error getting context for section '{section_heading}': {e}")
        return {}

def prune_sections(threshold: float = 0.5, max_age_days: int = 30):
    try:
        current_time = time.time()
        max_age_seconds = max_age_days * 24 * 60 * 60

        filter_condition = Filter(
            must=[
                FieldCondition(
                    key="metadata.relevance_score",
                    range=Range(lt=threshold)
                ),
                FieldCondition(
                    key="metadata.timestamp",
                    range=Range(lt=current_time - max_age_seconds)
                )
            ]
        )

        qdrant_client_readme.delete(
            collection_name=COLLECTION_NAME_README,
            filter=filter_condition
        )
        logger.info("Pruned low-relevance and old sections.")
    except Exception as e:
        logger.error(f"Error pruning sections: {e}")

# Initialize FastAPI app
app = FastAPI()

# Define Endpoints
@app.post("/process_readme")
async def process_readme_api(file: UploadFile = File(...)):
    try:
        content = await file.read()
        sections = parse_readme(content.decode())
        section_graph = build_section_graph(sections)
        for section in sections:
            section.vector = get_embedding(section.content).tolist()
        cluster_sections(sections)
        for section in sections:
            add_section_to_qdrant(section, section_graph)
        build_knn_index_readme()
        return {"message": "README processed successfully"}
    except Exception as e:
        logger.error(f"Error processing README: {e}")
        raise HTTPException(status_code=500, detail="Failed to process README.")

@app.post("/search")
async def search_api(query: str, top_k: int = 5):
    try:
        results = search_sections(query, top_k)
        return {"results": results}
    except Exception as e:
        logger.error(f"Error during search: {e}")
        raise HTTPException(status_code=500, detail="Search failed.")

@app.get("/context/{section_heading}")
async def get_context_api(section_heading: str, depth: int = 1):
    try:
        context = get_context(section_heading, depth)
        return {"context": context}
    except Exception as e:
        logger.error(f"Error retrieving context: {e}")
        raise HTTPException(status_code=500, detail="Failed to retrieve context.")

@app.post("/prune")
async def prune_api(threshold: float = 0.5, max_age_days: int = 30):
    try:
        prune_sections(threshold, max_age_days)
        return {"message": "Pruning completed successfully"}
    except Exception as e:
        logger.error(f"Error during pruning: {e}")
        raise HTTPException(status_code=500, detail="Pruning failed.")

@app.post("/rebuild_knn_index")
async def rebuild_knn_index_api():
    try:
        build_knn_index_readme()
        return {"message": "KNN index rebuilt successfully"}
    except Exception as e:
        logger.error(f"Error rebuilding KNN index: {e}")
        raise HTTPException(status_code=500, detail="Failed to rebuild KNN index.")

# Function to run Uvicorn server in a separate thread
def run_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    loop = asyncio.get_event_loop()
    loop.run_until_complete(server.serve())

# Start the server in a separate thread
server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

print("FastAPI server is running on http://0.0.0.0:8000")

# Example Usage of MemoryManager (Optional)
# You can interact with MemoryManager separately if needed

# Example: Creating a memory
# await memory_manager.create_memory(content="Sample memory content.", metadata={"tags": ["example", "test"], "reference_tags": ["example"]})

# Example: Recalling memories
# memories = await memory_manager.recall_memory(query_content="Sample query.")
# print(memories)

# Example: Pruning memories
# await memory_manager.prune_memories()

# Example: Purging all memories
# await memory_manager.purge_all_memories()

# Example: Recalling memories with metadata
# memories_with_metadata = await memory_manager.recall_memory_with_metadata(query_content="Sample query.", search_metadata={"tags": "example"})
# print(memories_with_metadata)

# Example: Deleting memories by metadata
# await memory_manager.delete_memories_by_metadata(metadata={"tags": "test"})


  Using cached protobuf-5.28.2-cp310-abi3-win_amd64.whl.metadata (592 bytes)
Using cached protobuf-5.28.2-cp310-abi3-win_amd64.whl (431 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3
  Rolling back uninstall of protobuf
  Moving to d:\users\nasan\anaconda3\envs\myenv\lib\site-packages\google\protobuf\__init__.py
   from C:\Users\nasan\AppData\Local\Temp\pip-uninstall-1we76qm2\__init__.py
  Moving to d:\users\nasan\anaconda3\envs\myenv\lib\site-packages\google\protobuf\__pycache__\__init__.cpython-310.pyc
   from C:\Users\nasan\AppData\Local\Temp\pip-uninstall-1we76qm2\__pycache__\__init__.cpython-310.pyc
  Moving to d:\users\nasan\anaconda3\envs\myenv\lib\site-packages\google\protobuf\__pycache__\any_pb2.cpython-310.pyc
   from C:\Users\nasan\AppData\Local\Temp\pip-uninstall-1we76qm2\__pycache__\any_pb2.cpython-310.pyc
  Moving to d:\users\nasan\anaconda3\envs\myenv\l

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\Users\\nasan\\anaconda3\\envs\\myenv\\Lib\\site-packages\\google\\_upb\\_message.pyd'
Consider using the `--user` option or check the permissions.

INFO:httpx:HTTP Request: GET http://localhost:6333/collections/advanced_readme_sections "HTTP/1.1 200 OK"
INFO:__main__:Collection 'advanced_readme_sections' already exists.
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Mind "HTTP/1.1 200 OK"
INFO:__main__:Collection 'Mind' already exists.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/Mind "HTTP/1.1 200 OK"
INFO:__main__:Collection 'Mind' exists.
INFO:__main__:Training XGBRanker is not implemented. Using default model.


FastAPI server is running on http://0.0.0.0:8000


INFO:     Started server process [35356]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


In [11]:
!pip install sentence_transformers

  Using cached sentence_transformers-3.1.1-py3-none-any.whl.metadata (10 kB)
Using cached sentence_transformers-3.1.1-py3-none-any.whl (245 kB)


In [ ]:
# Install Required Dependencies
!pip install fastapi uvicorn requests numpy qdrant-client markdown beautifulsoup4 scikit-learn xgboost networkx nest_asyncio python-dotenv sentence-transformers

# Comprehensive Implementation in One Code Block

import os
import uuid
import time
import math
import logging
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import asyncio
import requests
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import (
    Distance, VectorParams, PointStruct, Filter, FieldCondition, Range
)
from fastapi import FastAPI, HTTPException, UploadFile, File
from pydantic import BaseModel
import markdown
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from xgboost import XGBRanker
import networkx as nx
import nest_asyncio
import uvicorn
from threading import Thread
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Qdrant clients for both collections
qdrant_client_readme = QdrantClient(host="localhost", port=6333)
qdrant_client_mind = QdrantClient(host="localhost", port=6333)

# Constants for Readme Sections
COLLECTION_NAME_README = "advanced_readme_sections"
VECTOR_SIZE_README = 384  # Adjust based on your embedding model

# Constants for Memory Manager
COLLECTION_NAME_MIND = "Mind"
VECTOR_SIZE_MIND = 384  # Example size; will be updated based on model

# Create Readme Sections Collection if it doesn't exist
try:
    qdrant_client_readme.get_collection(COLLECTION_NAME_README)
    logger.info(f"Collection '{COLLECTION_NAME_README}' already exists.")
except Exception:
    logger.info(f"Creating collection '{COLLECTION_NAME_README}'.")
    qdrant_client_readme.create_collection(
        collection_name=COLLECTION_NAME_README,
        vectors_config=VectorParams(size=VECTOR_SIZE_README, distance=Distance.EUCLID)
    )

# Create Mind Collection if it doesn't exist
try:
    qdrant_client_mind.get_collection(COLLECTION_NAME_MIND)
    logger.info(f"Collection '{COLLECTION_NAME_MIND}' already exists.")
except Exception:
    logger.info(f"Creating collection '{COLLECTION_NAME_MIND}'.")
    # Initialize SentenceTransformer for MemoryManager
    memory_model = SentenceTransformer('all-MiniLM-L6-v2')
    VECTOR_SIZE_MIND = memory_model.get_sentence_embedding_dimension()
    qdrant_client_mind.create_collection(
        collection_name=COLLECTION_NAME_MIND,
        vectors_config=VectorParams(size=VECTOR_SIZE_MIND, distance=Distance.COSINE)
    )

# Define Data Models
@dataclass
class ReadmeSection:
    content: str
    heading: str
    level: int
    parent: Optional[str]
    children: List[str]
    metadata: Dict[str, Any]
    vector: Optional[List[float]] = None

class MemoryPacket(BaseModel):
    vector: List[float]
    content: str
    metadata: Dict[str, Any]

# Define MemoryManager Class
class MemoryManager:
    def __init__(self, qdrant_client: QdrantClient, collection_name: str, model_name: str = 'all-MiniLM-L6-v2'):
        self.qdrant_client = qdrant_client
        self.collection_name = collection_name
        self.model = SentenceTransformer(model_name)
        self._setup_collection()

    def _setup_collection(self):
        try:
            self.qdrant_client.get_collection(self.collection_name)
            logger.info(f"Collection '{self.collection_name}' exists.")
        except Exception:
            logger.info(f"Creating collection '{self.collection_name}'.")
            self.qdrant_client.create_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(size=self.model.get_sentence_embedding_dimension(), distance=Distance.COSINE)
            )

    async def create_memory(self, content: str, metadata: Dict[str, Any]):
        vector = self.model.encode(content).tolist()
        memory_packet = MemoryPacket(vector=vector, content=content, metadata=metadata)
        point_id = str(uuid.uuid4())

        try:
            self.qdrant_client.upsert(
                collection_name=self.collection_name,
                points=[PointStruct(id=point_id, vector=vector, payload=memory_packet.dict())]
            )
            logger.info(f"Memory created successfully with ID: {point_id}")
        except Exception as e:
            logger.error(f"Error creating memory: {e}")

    async def recall_memory(self, query_content: str, top_k: int = 5):
        query_vector = self.model.encode(query_content).tolist()

        try:
            results = self.qdrant_client.search(
                collection_name=self.collection_name,
                query_vector=query_vector,
                limit=top_k
            )

            memories = [MemoryPacket(**hit.payload) for hit in results]

            for memory in memories:
                self._update_relevance(memory, query_vector)

            ranked_memories = sorted(
                memories,
                key=lambda mem: (
                    mem.metadata['semantic_relativity'] * mem.metadata['memetic_similarity'] * mem.metadata['gravitational_pull']
                ),
                reverse=True
            )

            return [{
                "content": memory.content,
                "metadata": memory.metadata
            } for memory in ranked_memories[:top_k]]
        except Exception as e:
            logger.error(f"Error recalling memory: {e}")
            return []

    def _update_relevance(self, memory: MemoryPacket, query_vector: List[float]):
        memory.metadata["semantic_relativity"] = self._calculate_cosine_similarity(memory.vector, query_vector)
        memory.metadata["memetic_similarity"] = self._calculate_memetic_similarity(memory.metadata)
        memory.metadata["gravitational_pull"] = self._calculate_gravitational_pull(memory)
        memory.metadata["spacetime_coordinate"] = self._calculate_spacetime_coordinate(memory)

    @staticmethod
    def _calculate_cosine_similarity(vector_a: List[float], vector_b: List[float]) -> float:
        dot_product = sum(a * b for a, b in zip(vector_a, vector_b))
        magnitude_a = math.sqrt(sum(a ** 2 for a in vector_a))
        magnitude_b = math.sqrt(sum(b ** 2 for b in vector_b))

        if magnitude_a == 0 or magnitude_b == 0:
            return 0.0

        return dot_product / (magnitude_a * magnitude_b)

    @staticmethod
    def _calculate_memetic_similarity(metadata: Dict[str, Any]) -> float:
        tags = set(metadata.get("tags", []))
        reference_tags = set(metadata.get("reference_tags", []))

        if not tags or not reference_tags:
            return 1.0

        intersection = len(tags.intersection(reference_tags))
        union = len(tags.union(reference_tags))

        return intersection / union if union > 0 else 1.0

    @staticmethod
    def _calculate_gravitational_pull(memory: MemoryPacket) -> float:
        vector_magnitude = math.sqrt(sum(x ** 2 for x in memory.vector))
        recall_count = memory.metadata.get("recall_count", 0)
        memetic_similarity = memory.metadata.get("memetic_similarity", 1.0)
        semantic_relativity = memory.metadata.get("semantic_relativity", 1.0)

        return vector_magnitude * (1 + math.log1p(recall_count)) * memetic_similarity * semantic_relativity

    @staticmethod
    def _calculate_spacetime_coordinate(memory: MemoryPacket) -> float:
        time_decay_factor = 1 + (time.time() - memory.metadata.get("timestamp", time.time()))
        return memory.metadata["gravitational_pull"] / time_decay_factor

    async def prune_memories(self, threshold: float = 1e-5, max_age_days: int = 30):
        try:
            current_time = time.time()
            max_age_seconds = max_age_days * 24 * 60 * 60

            filter_condition = Filter(
                must=[
                    FieldCondition(
                        key="metadata.relevance_score",
                        range=Range(lt=threshold)
                    ),
                    FieldCondition(
                        key="metadata.timestamp",
                        range=Range(lt=current_time - max_age_seconds)
                    )
                ]
            )

            self.qdrant_client.delete(
                collection_name=self.collection_name,
                filter=filter_condition
            )
            logger.info("Pruned low-relevance and old memories.")
        except Exception as e:
            logger.error(f"Error pruning memories: {e}")

    async def purge_all_memories(self):
        try:
            self.qdrant_client.delete_collection(self.collection_name)
            self._setup_collection()
            logger.info(f"Purged all memories in the collection '{self.collection_name}'.")
        except Exception as e:
            logger.error(f"Error purging all memories: {e}")
            raise e

    async def recall_memory_with_metadata(self, query_content: str, search_metadata: Dict[str, Any], top_k: int = 10):
        try:
            query_vector = self.model.encode(query_content).tolist()
            results = self.qdrant_client.search(
                collection_name=self.collection_name,
                query_vector=query_vector,
                limit=top_k
            )

            memories = [MemoryPacket(**hit.payload) for hit in results]

            matching_memories = []
            for memory in memories:
                memory_metadata = memory.metadata
                if all(memory_metadata.get(key) == value for key, value in search_metadata.items()):
                    matching_memories.append({
                        "content": memory.content,
                        "metadata": memory_metadata
                    })

            if not matching_memories:
                return {"message": "No matching memories found"}

            return {"memories": matching_memories}
        except Exception as e:
            logger.error(f"Error recalling memories by metadata: {str(e)}")
            return {"message": "Error during memory recall"}

    async def delete_memories_by_metadata(self, metadata: Dict[str, Any]):
        try:
            # Scroll through all memories in the collection
            scroll_result = self.qdrant_client.scroll(self.collection_name, limit=1000)

            memories_to_delete = []
            for point in scroll_result:
                point_metadata = point.payload.get("metadata", {})
                if all(point_metadata.get(key) == value for key, value in metadata.items()):
                    memories_to_delete.append(point.id)

            if memories_to_delete:
                self.qdrant_client.delete(
                    collection_name=self.collection_name,
                    points_selector={"points": memories_to_delete}
                )
                logger.info(f"Deleted {len(memories_to_delete)} memories matching the metadata.")
            else:
                logger.info("No memories found matching the specified metadata.")
        except Exception as e:
            logger.error(f"Error deleting memories by metadata: {str(e)}")

# Initialize MemoryManager for Mind Collection
memory_manager = MemoryManager(
    qdrant_client=qdrant_client_mind,
    collection_name=COLLECTION_NAME_MIND,
    model_name='all-MiniLM-L6-v2'
)

# Utility Functions for Readme Processing

def get_embedding(text: str) -> np.ndarray:
    OLLAMA_API_URL = os.getenv("OLLAMA_API_URL", "http://localhost:11434/api/embeddings")
    try:
        response = requests.post(OLLAMA_API_URL, json={
            "model": "nomic-embed-text",
            "prompt": text
        })
        response.raise_for_status()
        return np.array(response.json()['embedding'])
    except Exception as e:
        logger.error(f"Error fetching embedding: {e}")
        raise

def parse_readme(content: str) -> List[ReadmeSection]:
    html = markdown.markdown(content)
    soup = BeautifulSoup(html, 'html.parser')
    sections = []
    section_stack = []
    current_section = None

    for elem in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'pre', 'ul', 'ol']):
        if elem.name.startswith('h'):
            level = int(elem.name[1])
            while section_stack and section_stack[-1].level >= level:
                section_stack.pop()

            parent = section_stack[-1] if section_stack else None
            current_section = ReadmeSection(
                content='',
                heading=elem.text.strip(),
                level=level,
                parent=parent.heading if parent else None,
                children=[],
                metadata={}
            )
            if parent:
                parent.children.append(current_section.heading)
            sections.append(current_section)
            section_stack.append(current_section)
        else:
            if current_section:
                current_section.content += "\n" + elem.get_text(separator=" ", strip=True)

    return sections

def build_section_graph(sections: List[ReadmeSection]) -> nx.DiGraph:
    G = nx.DiGraph()
    for section in sections:
        G.add_node(section.heading, level=section.level)
        if section.parent:
            G.add_edge(section.parent, section.heading)
    return G

def cluster_sections(sections: List[ReadmeSection], n_clusters: int = 10):
    embeddings = np.array([section.vector for section in sections if section.vector is not None])
    if embeddings.size == 0:
        logger.warning("No embeddings available for clustering.")
        return

    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings)
    for section, label in zip([s for s in sections if s.vector is not None], cluster_labels):
        section.metadata['cluster'] = int(label)

def add_section_to_qdrant(section: ReadmeSection, section_graph: nx.DiGraph):
    if not section.vector:
        logger.error(f"Section '{section.heading}' has no vector.")
        return

    point_id = str(uuid.uuid4())
    timestamp = time.time()

    centrality = nx.degree_centrality(section_graph).get(section.heading, 0)
    try:
        depth = nx.shortest_path_length(section_graph, source=list(section_graph.nodes)[0], target=section.heading)
    except nx.NetworkXNoPath:
        depth = 0

    payload = {
        "content": section.content,
        "heading": section.heading,
        "level": section.level,
        "parent": section.parent,
        "children": section.children,
        "metadata": {
            **section.metadata,
            "timestamp": timestamp,
            "centrality": centrality,
            "depth": depth,
            "access_count": 0,
            "relevance_score": 1.0
        }
    }

    try:
        qdrant_client_readme.upsert(
            collection_name=COLLECTION_NAME_README,
            points=[PointStruct(id=point_id, vector=section.vector, payload=payload)]
        )
        logger.info(f"Section '{section.heading}' added to Qdrant with ID {point_id}.")
    except Exception as e:
        logger.error(f"Failed to upsert section '{section.heading}': {e}")

knn_model_readme: Optional[NearestNeighbors] = None
point_id_mapping_readme: Dict[int, str] = {}

def build_knn_index_readme():
    global knn_model_readme, point_id_mapping_readme
    logger.info("Building KNN index for Readme Sections...")
    try:
        # Scroll retrieves points in batches; adjust batch size as needed
        all_points = []
        scroll_response = qdrant_client_readme.scroll(collection_name=COLLECTION_NAME_README, limit=10000)
        while scroll_response:
            all_points.extend(scroll_response.points)
            if scroll_response.next_page_offset:
                scroll_response = qdrant_client_readme.scroll(
                    collection_name=COLLECTION_NAME_README,
                    limit=10000,
                    offset=scroll_response.next_page_offset
                )
            else:
                break

        if not all_points:
            logger.warning("No points found in the Readme collection. KNN index not built.")
            knn_model_readme = None
            point_id_mapping_readme = {}
            return

        embeddings = np.array([point.vector for point in all_points])
        if embeddings.size == 0:
            logger.warning("Embeddings array is empty for Readme sections. KNN index not built.")
            knn_model_readme = None
            point_id_mapping_readme = {}
            return

        knn_model_readme = NearestNeighbors(n_neighbors=10, algorithm='auto', metric='euclidean')
        knn_model_readme.fit(embeddings)
        point_id_mapping_readme = {i: point.id for i, point in enumerate(all_points)}
        logger.info(f"KNN index for Readme sections built successfully with {len(point_id_mapping_readme)} points.")
    except Exception as e:
        logger.error(f"Error building KNN index for Readme sections: {e}")
        knn_model_readme = None
        point_id_mapping_readme = {}

tfidf_vectorizer = TfidfVectorizer()

def calculate_tfidf_similarity(query: str, document: str) -> float:
    tfidf_matrix = tfidf_vectorizer.fit_transform([query, document])
    return (tfidf_matrix * tfidf_matrix.T).A[0, 1]

def prepare_training_data(query: str, sections: List[Dict[str, Any]]):
    features = []
    labels = []
    for section in sections:
        feature_vector = [
            section['metadata'].get('tfidf_similarity', 0.0),
            section['metadata'].get('semantic_similarity', 0.0),
            section['metadata'].get('centrality', 0.0),
            section['level'],
            section['metadata'].get('cluster', 0)
        ]
        features.append(feature_vector)
        labels.append(section['metadata'].get('relevance_label', 1))  # Placeholder
    return np.array(features), np.array(labels)

xgb_ranker = XGBRanker(
    objective='rank:pairwise',
    learning_rate=0.1,
    max_depth=6,
    n_estimators=100
)

def train_xgb_ranker():
    try:
        # Placeholder: Implement actual training logic
        # This should be done offline with proper labeled data
        # For demonstration, we'll skip training
        logger.info("Training XGBRanker is not implemented. Using default model.")
    except Exception as e:
        logger.error(f"Error training XGBRanker: {e}")

# Train the ranker (currently a placeholder)
train_xgb_ranker()

def search_sections(query: str, top_k: int = 5) -> List[Dict[str, Any]]:
    if knn_model_readme is None:
        logger.warning("KNN model for Readme sections is not built. No search can be performed.")
        return []

    try:
        query_vector = get_embedding(query).reshape(1, -1)
    except Exception as e:
        logger.error(f"Failed to get embedding for query '{query}': {e}")
        return []

    try:
        distances, indices = knn_model_readme.kneighbors(query_vector)
    except Exception as e:
        logger.error(f"Error during KNN search: {e}")
        return []

    nearest_points = [point_id_mapping_readme[idx] for idx in indices[0]]

    sections = []
    for idx, point_id in enumerate(nearest_points):
        try:
            points = qdrant_client_readme.retrieve(collection_name=COLLECTION_NAME_README, ids=[point_id])
            if not points:
                continue
            point = points[0]
            section = point.payload
            section['vector'] = point.vector.tolist()
            tfidf_sim = calculate_tfidf_similarity(query, section['content'])
            section['metadata']['tfidf_similarity'] = tfidf_sim
            # Use the distance directly
            semantic_sim = 1 / (1 + distances[0][idx])
            section['metadata']['semantic_similarity'] = semantic_sim
            sections.append(section)
        except Exception as e:
            logger.error(f"Error retrieving section '{point_id}': {e}")

    if not sections:
        return []

    X_test, _ = prepare_training_data(query, sections)
    if X_test.size == 0:
        logger.warning("No features available for ranking.")
        return []

    try:
        relevance_scores = xgb_ranker.predict(X_test)
    except Exception as e:
        logger.error(f"Error during ranking: {e}")
        relevance_scores = np.ones(len(sections))  # Fallback

    for section, score in zip(sections, relevance_scores):
        section['score'] = score
    sections.sort(key=lambda x: x['score'], reverse=True)

    for section in sections[:top_k]:
        update_section_relevance(section['id'], section['score'])

    return sections[:top_k]

def update_section_relevance(point_id: str, score: float):
    try:
        points = qdrant_client_readme.retrieve(collection_name=COLLECTION_NAME_README, ids=[point_id])
        if not points:
            logger.warning(f"Point ID '{point_id}' not found for relevance update.")
            return
        current_payload = points[0].payload
        current_payload['metadata']['access_count'] += 1
        current_payload['metadata']['relevance_score'] = (
            current_payload['metadata']['relevance_score'] + score
        ) / 2

        qdrant_client_readme.upsert(
            collection_name=COLLECTION_NAME_README,
            points=[PointStruct(id=point_id, vector=points[0].vector.tolist(), payload=current_payload)]
        )
        logger.info(f"Updated relevance for point ID {point_id}.")
    except Exception as e:
        logger.error(f"Error updating relevance for point ID '{point_id}': {e}")

def get_context(section_heading: str, depth: int = 1) -> Dict[str, Any]:
    try:
        filter_condition = Filter(
            must=[FieldCondition(key="heading", match={'value': section_heading})]
        )
        results = qdrant_client_readme.scroll(
            collection_name=COLLECTION_NAME_README,
            filter=filter_condition,
            limit=1
        )
        if not results.points:
            return {}

        section = results.points[0].payload
        context = {
            "current": section,
            "parent": None,
            "children": [],
            "siblings": []
        }

        if section.get('parent'):
            parent_filter = Filter(
                must=[FieldCondition(key="heading", match={'value': section['parent']})]
            )
            parent_results = qdrant_client_readme.scroll(
                collection_name=COLLECTION_NAME_README,
                filter=parent_filter,
                limit=1
            )
            if parent_results.points:
                context["parent"] = parent_results.points[0].payload

        if depth > 0 and 'children' in section:
            for child_heading in section['children']:
                child_context = get_context(child_heading, depth - 1)
                if child_context:
                    context["children"].append(child_context["current"])

        if context.get("parent") and 'children' in context["parent"]:
            for sibling_heading in context["parent"]["children"]:
                if sibling_heading != section_heading:
                    sibling_context = get_context(sibling_heading, 0)
                    if sibling_context:
                        context["siblings"].append(sibling_context["current"])

        return context
    except Exception as e:
        logger.error(f"Error getting context for section '{section_heading}': {e}")
        return {}

def prune_sections(threshold: float = 0.5, max_age_days: int = 30):
    try:
        current_time = time.time()
        max_age_seconds = max_age_days * 24 * 60 * 60

        filter_condition = Filter(
            must=[
                FieldCondition(
                    key="metadata.relevance_score",
                    range=Range(lt=threshold)
                ),
                FieldCondition(
                    key="metadata.timestamp",
                    range=Range(lt=current_time - max_age_seconds)
                )
            ]
        )

        qdrant_client_readme.delete(
            collection_name=COLLECTION_NAME_README,
            filter=filter_condition
        )
        logger.info("Pruned low-relevance and old sections.")
    except Exception as e:
        logger.error(f"Error pruning sections: {e}")

# Initialize FastAPI app
app = FastAPI()

# Define Endpoints
@app.post("/process_readme")
async def process_readme_api(file: UploadFile = File(...)):
    try:
        content = await file.read()
        sections = parse_readme(content.decode())
        section_graph = build_section_graph(sections)
        for section in sections:
            section.vector = get_embedding(section.content).tolist()
        cluster_sections(sections)
        for section in sections:
            add_section_to_qdrant(section, section_graph)
        build_knn_index_readme()
        return {"message": "README processed successfully"}
    except Exception as e:
        logger.error(f"Error processing README: {e}")
        raise HTTPException(status_code=500, detail="Failed to process README.")

@app.post("/search")
async def search_api(query: str, top_k: int = 5):
    try:
        results = search_sections(query, top_k)
        return {"results": results}
    except Exception as e:
        logger.error(f"Error during search: {e}")
        raise HTTPException(status_code=500, detail="Search failed.")

@app.get("/context/{section_heading}")
async def get_context_api(section_heading: str, depth: int = 1):
    try:
        context = get_context(section_heading, depth)
        return {"context": context}
    except Exception as e:
        logger.error(f"Error retrieving context: {e}")
        raise HTTPException(status_code=500, detail="Failed to retrieve context.")

@app.post("/prune")
async def prune_api(threshold: float = 0.5, max_age_days: int = 30):
    try:
        prune_sections(threshold, max_age_days)
        return {"message": "Pruning completed successfully"}
    except Exception as e:
        logger.error(f"Error during pruning: {e}")
        raise HTTPException(status_code=500, detail="Pruning failed.")

@app.post("/rebuild_knn_index")
async def rebuild_knn_index_api():
    try:
        build_knn_index_readme()
        return {"message": "KNN index rebuilt successfully"}
    except Exception as e:
        logger.error(f"Error rebuilding KNN index: {e}")
        raise HTTPException(status_code=500, detail="Failed to rebuild KNN index.")

# Function to run Uvicorn server in a separate thread
def run_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    loop = asyncio.get_event_loop()
    loop.run_until_complete(server.serve())

# Start the server in a separate thread
server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

print("FastAPI server is running on http://0.0.0.0:8000")

# Example Usage of MemoryManager (Optional)
# You can interact with MemoryManager separately if needed

# Example: Creating a memory
# await memory_manager.create_memory(content="Sample memory content.", metadata={"tags": ["example", "test"], "reference_tags": ["example"]})

# Example: Recalling memories
# memories = await memory_manager.recall_memory(query_content="Sample query.")
# print(memories)

# Example: Pruning memories
# await memory_manager.prune_memories()

# Example: Purging all memories
# await memory_manager.purge_all_memories()

# Example: Recalling memories with metadata
# memories_with_metadata = await memory_manager.recall_memory_with_metadata(query_content="Sample query.", search_metadata={"tags": "example"})
# print(memories_with_metadata)

# Example: Deleting memories by metadata
# await memory_manager.delete_memories_by_metadata(metadata={"tags": "test"})


In [5]:
# Install Required Dependencies
# !pip install fastapi uvicorn requests numpy qdrant-client markdown beautifulsoup4 scikit-learn xgboost networkx nest_asyncio python-dotenv sentence-transformers

# Comprehensive Implementation

import os
import uuid
import time
import math
import logging
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import asyncio
import requests
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import (
    Distance, VectorParams, PointStruct, Filter, FieldCondition, Range, MatchValue
)
from fastapi import FastAPI, HTTPException, UploadFile, File
from pydantic import BaseModel
import markdown
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from xgboost import XGBRanker
import networkx as nx
import nest_asyncio
import uvicorn
from threading import Thread
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Qdrant client
qdrant_client = QdrantClient(host="localhost", port=6333)

# Constants
COLLECTION_NAME = "advanced_readme_sections"
VECTOR_SIZE = 384  # Adjust based on your embedding model

# Initialize SentenceTransformer
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
VECTOR_SIZE = embedding_model.get_sentence_embedding_dimension()

# Create Collection if it doesn't exist
try:
    qdrant_client.get_collection(COLLECTION_NAME)
    logger.info(f"Collection '{COLLECTION_NAME}' already exists.")
except Exception:
    logger.info(f"Creating collection '{COLLECTION_NAME}'.")
    qdrant_client.recreate_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE)
    )

# Define Data Models
@dataclass
class ReadmeSection:
    content: str
    heading: str
    level: int
    parent: Optional[str]
    children: List[str]
    metadata: Dict[str, Any]
    vector: Optional[List[float]] = None

# Function to send request with metadata
import json
baseUrl = "http://localhost:8000"

def send_request_with_metadata(title, metadata):
    url = f"{baseUrl}/create_memory"
    # Define headers
    headers = {
        "Content-Type": "application/json"
    }
    # Define the body
    body = {
        "content": title,
        "metadata": metadata
    }
    # Send the POST request
    try:
        response = requests.post(
            url,
            headers=headers,
            data=json.dumps(body)  # Convert the body to JSON format
        )
        
        # Check if the request was successful
        if response.status_code == 200:
            return response.json()  # Return the response as JSON
        else:
            logger.error(f"Request failed with status code {response.status_code}: {response.text}")
            return {"error": f"Request failed with status code {response.status_code}", "details": response.text}
    
    except requests.RequestException as e:
        logger.error(f"Request failed: {e}")
        return {"error": "Request failed", "details": str(e)}

# Utility Functions for Readme Processing

def parse_readme(content: str) -> List[ReadmeSection]:
    html = markdown.markdown(content)
    soup = BeautifulSoup(html, 'html.parser')
    sections = []
    section_stack = []
    current_section = None

    for elem in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'pre', 'ul', 'ol']):
        if elem.name.startswith('h'):
            level = int(elem.name[1])
            while section_stack and section_stack[-1].level >= level:
                section_stack.pop()

            parent = section_stack[-1] if section_stack else None
            current_section = ReadmeSection(
                content='',
                heading=elem.text.strip(),
                level=level,
                parent=parent.heading if parent else None,
                children=[],
                metadata={}
            )
            if parent:
                parent.children.append(current_section.heading)
            sections.append(current_section)
            section_stack.append(current_section)
        else:
            if current_section:
                current_section.content += "\n" + elem.get_text(separator=" ", strip=True)

    return sections

def build_section_graph(sections: List[ReadmeSection]) -> nx.DiGraph:
    G = nx.DiGraph()
    for section in sections:
        G.add_node(section.heading, level=section.level)
        if section.parent:
            G.add_edge(section.parent, section.heading)
    return G

def add_section_to_qdrant(section: ReadmeSection):
    if not section.vector:
        logger.error(f"Section '{section.heading}' has no vector.")
        return

    point_id = str(uuid.uuid4())
    timestamp = time.time()

    payload = {
        "content": section.content,
        "heading": section.heading,
        "level": section.level,
        "parent": section.parent,
        "children": section.children,
        "metadata": {
            **section.metadata,
            "timestamp": timestamp,
            "access_count": 0,
            "relevance_score": 1.0
        }
    }

    try:
        qdrant_client.upsert(
            collection_name=COLLECTION_NAME,
            points=[PointStruct(id=point_id, vector=section.vector, payload=payload)]
        )
        logger.info(f"Section '{section.heading}' added to Qdrant with ID {point_id}.")
    except Exception as e:
        logger.error(f"Failed to upsert section '{section.heading}': {e}")

def get_context(section_heading: str, depth: int = 1) -> Dict[str, Any]:
    try:
        filter_condition = Filter(
            must=[FieldCondition(key="heading", match=MatchValue(value=section_heading))]
        )
        results = qdrant_client.scroll(
            collection_name=COLLECTION_NAME,
            scroll_filter=filter_condition,
            limit=1
        )
        if not results:
            return {}

        section = results[0].payload
        context = {
            "current": section,
            "parent": None,
            "children": [],
            "siblings": []
        }

        if section.get('parent'):
            parent_filter = Filter(
                must=[FieldCondition(key="heading", match=MatchValue(value=section['parent']))]
            )
            parent_results = qdrant_client.scroll(
                collection_name=COLLECTION_NAME,
                scroll_filter=parent_filter,
                limit=1
            )
            if parent_results:
                context["parent"] = parent_results[0].payload

        if depth > 0 and 'children' in section:
            for child_heading in section['children']:
                child_context = get_context(child_heading, depth - 1)
                if child_context:
                    context["children"].append(child_context["current"])

        if context.get("parent") and 'children' in context["parent"]:
            for sibling_heading in context["parent"]["children"]:
                if sibling_heading != section_heading:
                    sibling_context = get_context(sibling_heading, 0)
                    if sibling_context:
                        context["siblings"].append(sibling_context["current"])

        return context
    except Exception as e:
        logger.error(f"Error getting context for section '{section_heading}': {e}")
        return {}

# Initialize FastAPI app
app = FastAPI()

# Define Endpoints
@app.post("/create_memory")
async def create_memory_api(content: str, metadata: Dict[str, Any]):
    try:
        vector = embedding_model.encode(content).tolist()
        point_id = str(uuid.uuid4())
        payload = {
            "content": content,
            "metadata": metadata
        }
        qdrant_client.upsert(
            collection_name=COLLECTION_NAME,
            points=[PointStruct(id=point_id, vector=vector, payload=payload)]
        )
        logger.info(f"Memory created successfully with ID: {point_id}")
        return {"message": "Memory created successfully", "id": point_id}
    except Exception as e:
        logger.error(f"Error creating memory: {e}")
        raise HTTPException(status_code=500, detail="Failed to create memory.")

# Function to run Uvicorn server in a separate thread
def run_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    loop = asyncio.get_event_loop()
    loop.run_until_complete(server.serve())

# Start the server in a separate thread
server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

print("FastAPI server is running on http://0.0.0.0:8000")

# Function to process README and send sections to database
def process_readme_and_send(readme_path: str):
    with open(readme_path, 'r', encoding='utf-8') as f:
        content = f.read()
    sections = parse_readme(content)
    section_graph = build_section_graph(sections)
    for section in sections:
        # Generate embedding
        section.vector = embedding_model.encode(section.content).tolist() if section.content else None

        # Prepare title and metadata
        title = section.heading
        metadata = {
            "content": section.content,
            "level": section.level,
            "parent": section.parent,
            "children": section.children,
            "metadata": {
                **section.metadata,
                "timestamp": time.time(),
                "access_count": 0,
                "relevance_score": 1.0
            }
        }

        # Send to database
        response = send_request_with_metadata(title, metadata)
        print(f"Sent section '{title}' to database. Response: {response}")

        # Optionally, add to Qdrant directly
        add_section_to_qdrant(section)


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:httpx:HTTP Request: GET http://localhost:6333/collections/advanced_readme_sections "HTTP/1.1 200 OK"
INFO:__main__:Collection 'advanced_readme_sections' already exists.


FastAPI server is running on http://0.0.0.0:8000


INFO:     Started server process [44932]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.


INFO:     Application shutdown complete.


In [5]:
# Test Code

# Sample README content (you can replace this with the path to your actual README file)
sample_readme_content = """
# Sample Project

## Introduction

This is a sample README file for testing purposes.

## Installation

Instructions to install...

### Prerequisites

List of prerequisites...

### Step-by-Step Guide

Step-by-step installation guide...

## Usage

How to use the project...

"""

# Save the sample README content to a file
readme_path = "sample_README.md"
with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(sample_readme_content)

# Wait for the server to start
import time
time.sleep(2)

# Process the README and send sections to the database
process_readme_and_send(readme_path)

# Retrieve context for a section
section_heading = "Installation"
context = get_context(section_heading, depth=1)

# Print the context
import json
print("\nRetrieved Context:")
print(json.dumps(context, indent=2))


ERROR:__main__:Request failed with status code 422: {"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}
ERROR:__main__:Section 'Sample Project' has no vector.


Sent section 'Sample Project' to database. Response: {'error': 'Request failed with status code 422', 'details': '{"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}'}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:__main__:Request failed with status code 422: {"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/advanced_readme_sections/points?wait=true "HTTP/1.1 200 OK"
INFO:__main__:Section 'Introduction' added to Qdrant with ID 9b629c23-2901-4794-b8ca-d58bebb5fb3f.


Sent section 'Introduction' to database. Response: {'error': 'Request failed with status code 422', 'details': '{"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}'}


...

ERROR:__main__:Request failed with status code 422: {"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/advanced_readme_sections/points?wait=true "HTTP/1.1 200 OK"
INFO:__main__:Section 'Step-by-Step Guide' added to Qdrant with ID cb764f5c-c5e6-4d83-89d0-a17dc28991e1.


Sent section 'Step-by-Step Guide' to database. Response: {'error': 'Request failed with status code 422', 'details': '{"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}'}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:__main__:Request failed with status code 422: {"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/advanced_readme_sections/points?wait=true "HTTP/1.1 200 OK"
INFO:__main__:Section 'Usage' added to Qdrant with ID befe5248-b91c-4df2-a05d-1cc71eabef78.
INFO:httpx:HTTP Request: POST http://localhost:6333/collections/advanced_readme_sections/points/scroll "HTTP/1.1 200 OK"
ERROR:__main__:Error getting context for section 'Installation': 'list' object has no attribute 'payload'


Sent section 'Usage' to database. Response: {'error': 'Request failed with status code 422', 'details': '{"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}'}

Retrieved Context:
{}


In [6]:
# Install Required Dependencies
# You can run this in your terminal or uncomment the following line if using Jupyter.
# !pip install fastapi uvicorn requests numpy qdrant-client markdown beautifulsoup4 scikit-learn xgboost networkx nest_asyncio python-dotenv sentence-transformers

# Comprehensive Implementation

import os
import uuid
import time
import math
import logging
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
import asyncio
import requests
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.http.models import (
    Distance, VectorParams, PointStruct, Filter, FieldCondition, Range, MatchValue
)
from fastapi import FastAPI, HTTPException, UploadFile, File
from pydantic import BaseModel
import markdown
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from xgboost import XGBRanker
import networkx as nx
import nest_asyncio
import uvicorn
from threading import Thread
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime

# Apply nest_asyncio to allow nested event loops in environments like Jupyter
nest_asyncio.apply()

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Qdrant client
qdrant_client = QdrantClient(host="localhost", port=6333)

# Define the collection name
COLLECTION_NAME = "advanced_readme_sections"

# Initialize SentenceTransformer for embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
VECTOR_SIZE = embedding_model.get_sentence_embedding_dimension()

# Create Collection if it doesn't exist, else verify vector size
def initialize_qdrant_collection():
    try:
        collection = qdrant_client.get_collection(COLLECTION_NAME)
        if collection.vectors.size != VECTOR_SIZE:
            logger.info(f"Existing collection '{COLLECTION_NAME}' has different vector size. Recreating collection.")
            qdrant_client.recreate_collection(
                collection_name=COLLECTION_NAME,
                vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE)
            )
            logger.info(f"Collection '{COLLECTION_NAME}' recreated with vector size {VECTOR_SIZE}.")
        else:
            logger.info(f"Collection '{COLLECTION_NAME}' already exists with correct vector size.")
    except Exception:
        logger.info(f"Creating collection '{COLLECTION_NAME}' with vector size {VECTOR_SIZE}.")
        qdrant_client.create_collection(
            collection_name=COLLECTION_NAME,
            vectors_config=VectorParams(size=VECTOR_SIZE, distance=Distance.COSINE)
        )
        logger.info(f"Collection '{COLLECTION_NAME}' created successfully.")

# Initialize the Qdrant collection
initialize_qdrant_collection()

# Define Data Models
@dataclass
class ReadmeSection:
    content: str
    heading: str
    level: int
    parent: Optional[str]
    children: List[str]
    metadata: Dict[str, Any]
    vector: Optional[List[float]] = None

class CreateMemoryRequest(BaseModel):
    content: str
    metadata: Dict[str, Any]

class CreateMemoryResponse(BaseModel):
    message: str
    id: Optional[str] = None

class SearchRequest(BaseModel):
    query: str
    top_k: int = 5

class SearchResult(BaseModel):
    content: str
    metadata: Dict[str, Any]
    score: float

class SearchResponse(BaseModel):
    results: List[SearchResult]

class ContextResponse(BaseModel):
    context: Dict[str, Any]

class PruneRequest(BaseModel):
    threshold: float = 0.5
    max_age_days: int = 30

class PruneResponse(BaseModel):
    message: str

class RebuildKNNResponse(BaseModel):
    message: str

# Initialize FastAPI app
app = FastAPI()

# Define Endpoints

@app.post("/create_memory", response_model=CreateMemoryResponse)
async def create_memory_api(request: CreateMemoryRequest):
    try:
        vector = embedding_model.encode(request.content).tolist()
        point_id = str(uuid.uuid4())
        payload = {
            "content": request.content,
            "metadata": request.metadata
        }
        qdrant_client.upsert(
            collection_name=COLLECTION_NAME,
            points=[PointStruct(id=point_id, vector=vector, payload=payload)]
        )
        logger.info(f"Memory created successfully with ID: {point_id}")
        return CreateMemoryResponse(message="Memory created successfully", id=point_id)
    except Exception as e:
        logger.error(f"Error creating memory: {e}")
        raise HTTPException(status_code=500, detail="Failed to create memory.")

@app.post("/search", response_model=SearchResponse)
async def search_api(request: SearchRequest):
    try:
        query_vector = embedding_model.encode(request.query).tolist()
        search_results = qdrant_client.search(
            collection_name=COLLECTION_NAME,
            query_vector=query_vector,
            limit=request.top_k
        )
        results = []
        for hit in search_results:
            # Calculate cosine similarity as semantic similarity
            semantic_sim = cosine_similarity([query_vector], [hit.vector])[0][0]
            # For demonstration, relevance_score is set as semantic_sim
            relevance_score = semantic_sim
            results.append(SearchResult(
                content=hit.payload["content"],
                metadata=hit.payload["metadata"],
                score=relevance_score
            ))
        # Sort results by score descending
        results_sorted = sorted(results, key=lambda x: x.score, reverse=True)
        return SearchResponse(results=results_sorted)
    except Exception as e:
        logger.error(f"Error during search: {e}")
        raise HTTPException(status_code=500, detail="Search failed.")

@app.get("/context/{section_heading}", response_model=ContextResponse)
async def get_context_api(section_heading: str, depth: int = 1):
    try:
        filter_condition = Filter(
            must=[FieldCondition(key="heading", match=MatchValue(value=section_heading))]
        )
        results = qdrant_client.scroll(
            collection_name=COLLECTION_NAME,
            scroll_filter=filter_condition,
            limit=1
        )
        if not results:
            raise HTTPException(status_code=404, detail="Section not found.")
        
        section = results[0].payload
        context = {
            "current": section,
            "parent": None,
            "children": [],
            "siblings": []
        }
        
        # Fetch parent section
        parent_heading = section.get("parent")
        if parent_heading:
            parent_filter = Filter(
                must=[FieldCondition(key="heading", match=MatchValue(value=parent_heading))]
            )
            parent_results = qdrant_client.scroll(
                collection_name=COLLECTION_NAME,
                scroll_filter=parent_filter,
                limit=1
            )
            if parent_results:
                context["parent"] = parent_results[0].payload
        
        # Fetch children sections
        children_headings = section.get("children", [])
        for child_heading in children_headings:
            child_filter = Filter(
                must=[FieldCondition(key="heading", match=MatchValue(value=child_heading))]
            )
            child_results = qdrant_client.scroll(
                collection_name=COLLECTION_NAME,
                scroll_filter=child_filter,
                limit=1
            )
            if child_results:
                context["children"].append(child_results[0].payload)
        
        # Fetch siblings
        if parent_heading:
            sibling_headings = context["parent"].get("children", [])
            for sibling_heading in sibling_headings:
                if sibling_heading != section_heading:
                    sibling_filter = Filter(
                        must=[FieldCondition(key="heading", match=MatchValue(value=sibling_heading))]
                    )
                    sibling_results = qdrant_client.scroll(
                        collection_name=COLLECTION_NAME,
                        scroll_filter=sibling_filter,
                        limit=1
                    )
                    if sibling_results:
                        context["siblings"].append(sibling_results[0].payload)
        
        return ContextResponse(context=context)
    except HTTPException as he:
        raise he
    except Exception as e:
        logger.error(f"Error retrieving context for section '{section_heading}': {e}")
        raise HTTPException(status_code=500, detail="Failed to retrieve context.")

@app.post("/prune", response_model=PruneResponse)
async def prune_api(prune_request: PruneRequest):
    try:
        current_time = time.time()
        max_age_seconds = prune_request.max_age_days * 24 * 60 * 60

        # Define filter for low relevance_score and older than max_age_days
        filter_condition = Filter(
            must=[
                FieldCondition(
                    key="metadata.relevance_score",
                    range=Range(lt=prune_request.threshold)
                ),
                FieldCondition(
                    key="metadata.timestamp",
                    range=Range(lt=current_time - max_age_seconds)
                )
            ]
        )

        qdrant_client.delete(
            collection_name=COLLECTION_NAME,
            filter=filter_condition
        )
        logger.info("Pruned low-relevance and old sections.")
        return PruneResponse(message="Pruning completed successfully")
    except Exception as e:
        logger.error(f"Error during pruning: {e}")
        raise HTTPException(status_code=500, detail="Pruning failed.")

@app.post("/rebuild_knn_index", response_model=RebuildKNNResponse)
async def rebuild_knn_index_api():
    try:
        # For Qdrant, the index is handled internally, so no action needed
        # If you have a separate KNN model, rebuild it here
        logger.info("KNN index rebuilt successfully (handled internally by Qdrant).")
        return RebuildKNNResponse(message="KNN index rebuilt successfully")
    except Exception as e:
        logger.error(f"Error rebuilding KNN index: {e}")
        raise HTTPException(status_code=500, detail="Failed to rebuild KNN index.")

# Function to run Uvicorn server in a separate thread
def run_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    loop = asyncio.get_event_loop()
    loop.run_until_complete(server.serve())

# Function to start the server
def start_server():
    server_thread = Thread(target=run_server, daemon=True)
    server_thread.start()
    logger.info("FastAPI server is running on http://0.0.0.0:8000")

# Initialize and start the server
start_server()

# Function to process README and send sections to database
def process_readme_and_send(readme_path: str):
    try:
        with open(readme_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except FileNotFoundError:
        logger.error(f"README file not found at path: {readme_path}")
        return
    
    sections = parse_readme(content)
    section_graph = build_section_graph(sections)
    
    for section in sections:
        if section.content.strip():
            # Generate embedding
            section.vector = embedding_model.encode(section.content).tolist()
        else:
            section.vector = None  # Handle sections with no content
    
        # Prepare title and metadata
        title = section.heading
        metadata = {
            "content": section.content,
            "level": section.level,
            "parent": section.parent,
            "children": section.children,
            "metadata": {
                "timestamp": int(time.time()),
                "access_count": 0,
                "relevance_score": 1.0,
                "cluster": section.metadata.get('cluster', 0),
                "centrality": 0.0  # Placeholder, can be calculated if needed
            }
        }
    
        if not section.vector:
            logger.warning(f"Section '{title}' has no content and will not be sent to the database.")
            continue
    
        # Send to database via API
        response = send_request_with_metadata(title, metadata)
        print(f"Sent section '{title}' to database. Response: {response}")
    
    logger.info("Finished processing and sending all sections.")

# Initialize a sample KNN model (if needed)
# Currently, Qdrant handles KNN internally

# Example: Function to retrieve context (already defined in API)

# Note: The server is running in the background as a daemon thread and will terminate when the main program exits.


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-8' coro=<Server.serve() done, defined at d:\Users\nasan\anaconda3\envs\myenv\lib\site-packages\uvicorn\server.py:63> exception=SystemExit(1)>
Traceback (most recent call last):
  File "d:\Users\nasan\anaconda3\envs\myenv\lib\site-packages\uvicorn\server.py", line 160, in startup
    server = await loop.create_server(
  File "d:\Users\nasan\anaconda3\envs\myenv\lib\asyncio\base_events.py", line 1519, in create_server
    raise OSError(err.errno, 'error while attempting '
OSError: [Errno 10048] error while attempting to bind on address ('0.0.0.0', 8000): only one usage of each socket address (protocol/network address/port) is normally permitted

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "d:\Users\nasan\anaconda3\envs\myenv\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\nasan\AppData\Roaming\Python\Python

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `advanced_readme_sections` already exists!"},"time":0.00003676}'

In [17]:
!pip install fastapi uvicorn qdrant-client markdown beautifulsoup4 sklearn xgboost networkx nest_asyncio sentence-transformers numpy requests

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-pypi-packag

: 

In [8]:
# Test Code

import os
import time
import json
import requests

# Define the path for the sample README
readme_path = "README.md"

# Sample README content
sample_readme_content = """
# Sample Project

## Introduction

This is a sample README file for testing purposes.

## Installation

Instructions to install...

### Prerequisites

List of prerequisites...

### Step-by-Step Guide

Step-by-step installation guide...

## Usage

How to use the project...
"""

# Write the sample README content to the file
with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(sample_readme_content)

# Wait briefly to ensure the server is up
time.sleep(3)  # Adjust if necessary based on server startup time

# Function to send POST request to /create_memory
def send_create_memory(title, metadata):
    url = "http://localhost:8000/create_memory"
    headers = {"Content-Type": "application/json"}
    body = {
        "content": title,
        "metadata": metadata
    }
    try:
        response = requests.post(url, headers=headers, json=body)
        return response.json()
    except Exception as e:
        return {"error": str(e)}

# Process the README and send sections to the database
from threading import Thread

def run_processing():
    process_readme_and_send(readme_path)

processing_thread = Thread(target=run_processing)
processing_thread.start()

# Wait for processing to complete
processing_thread.join()

# Function to retrieve context for a section
def get_section_context(section_heading, depth=1):
    url = f"http://localhost:8000/context/{section_heading}"
    params = {"depth": depth}
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Failed to retrieve context: {response.status_code}", "details": response.text}
    except Exception as e:
        return {"error": str(e)}

# Retrieve context for the 'Installation' section
context_response = get_section_context("Installation", depth=1)
print("\nRetrieved Context for 'Installation':")
print(json.dumps(context_response, indent=2))

# Function to perform a search
def perform_search(query, top_k=5):
    url = "http://localhost:8000/search"
    headers = {"Content-Type": "application/json"}
    body = {
        "query": query,
        "top_k": top_k
    }
    try:
        response = requests.post(url, headers=headers, json=body)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Search failed: {response.status_code}", "details": response.text}
    except Exception as e:
        return {"error": str(e)}

# Perform a search for "How to install dependencies"
search_query = "How to install dependencies"
search_response = perform_search(search_query, top_k=3)
print(f"\nSearch Results for '{search_query}':")
print(json.dumps(search_response, indent=2))

# Function to prune sections (optional)
def prune_sections(threshold=0.5, max_age_days=30):
    url = "http://localhost:8000/prune"
    headers = {"Content-Type": "application/json"}
    body = {
        "threshold": threshold,
        "max_age_days": max_age_days
    }
    try:
        response = requests.post(url, headers=headers, json=body)
        return response.json()
    except Exception as e:
        return {"error": str(e)}

# Optionally, prune sections
# prune_response = prune_sections()
# print(f"\nPrune Response: {prune_response}")

# Function to rebuild KNN index (optional)
def rebuild_knn():
    url = "http://localhost:8000/rebuild_knn_index"
    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(url, headers=headers)
        return response.json()
    except Exception as e:
        return {"error": str(e)}

# Optionally, rebuild KNN index
# rebuild_response = rebuild_knn()
# print(f"\nRebuild KNN Index Response: {rebuild_response}")

# Clean up: Remove the sample README file
# os.remove(readme_path)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:__main__:Section 'Sample Project' has no vector.


Sent section 'Sample Project' to database. Response: {'message': 'Memory created successfully', 'id': '0ba22cf0-adc7-4ffe-a7dd-865eebaac104'}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/advanced_readme_sections/points?wait=true "HTTP/1.1 200 OK"
INFO:__main__:Section 'Usage' added to Qdrant with ID a7614a63-35ea-4dbd-8149-2e0d8e71347e.


Sent section 'Usage' to database. Response: {'message': 'Memory created successfully', 'id': 'dcbda5b9-28db-4968-9350-1dcf062d566c'}

Retrieved Context for 'Installation':
{
  "error": "Failed to retrieve context: 500",
  "details": "{\"detail\":\"Failed to retrieve context.\"}"
}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Search Results for 'How to install dependencies':
{
  "error": "Search failed: 500",
  "details": "{\"detail\":\"Search failed.\"}"
}


In [10]:
import os
import time
import json
import requests

# Define the path for the actual README (replace with the correct path)
readme_path = "README.md"  # Make sure this points to your actual README

# Wait briefly to ensure the server is up (optional adjustment if necessary)
time.sleep(3)

# Function to send POST request to /create_memory
def send_create_memory(title, metadata):
    url = "http://localhost:8000/create_memory"
    headers = {"Content-Type": "application/json"}
    body = {
        "content": title,
        "metadata": metadata
    }
    try:
        response = requests.post(url, headers=headers, json=body)
        return response.json()
    except Exception as e:
        return {"error": str(e)}

# Process the README and send sections to the database
from threading import Thread

def process_readme_and_send(readme_path):
    try:
        with open(readme_path, 'r', encoding='utf-8') as f:
            readme_content = f.read()

        # Use the actual processing function here to break down the README content
        sections = parse_readme(readme_content)  # Assuming parse_readme function processes your README correctly

        # Build the section graph (for context of parent-child relationships)
        section_graph = build_section_graph(sections)

        # Generate embeddings and metadata for each section
        for section in sections:
            if section.content:
                section.vector = get_embedding(section.content).tolist()  # Get vector for the section
                add_section_to_qdrant(section, section_graph)  # Add to database
                print(f"Processed and added section '{section.heading}' to the database.")
            else:
                print(f"Skipping section '{section.heading}' due to missing content.")
        
        print("README processing completed.")
        
    except Exception as e:
        print(f"Error processing README: {str(e)}")

# Run the processing in a thread (optional, if necessary to avoid blocking)
processing_thread = Thread(target=process_readme_and_send, args=(readme_path,))
processing_thread.start()

# Wait for processing to complete
processing_thread.join()

# Function to retrieve context for a section
def get_section_context(section_heading, depth=1):
    url = f"http://localhost:8000/context/{section_heading}"
    params = {"depth": depth}
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Failed to retrieve context: {response.status_code}", "details": response.text}
    except Exception as e:
        return {"error": str(e)}

# Retrieve context for a specific section, e.g., 'Installation'
context_response = get_section_context("Installation", depth=1)
print("\nRetrieved Context for 'Installation':")
print(json.dumps(context_response, indent=2))

# Function to perform a search
def perform_search(query, top_k=5):
    url = "http://localhost:8000/search"
    headers = {"Content-Type": "application/json"}
    body = {
        "query": query,
        "top_k": top_k
    }
    try:
        response = requests.post(url, headers=headers, json=body)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Search failed: {response.status_code}", "details": response.text}
    except Exception as e:
        return {"error": str(e)}

# Perform a search query, for example "How to install dependencies"
search_query = "How to install dependencies"
search_response = perform_search(search_query, top_k=3)
print(f"\nSearch Results for '{search_query}':")
print(json.dumps(search_response, indent=2))

# Note: We have disabled any pruning and unnecessary operations to ensure the README is not altered in any way.


Error processing README: add_section_to_qdrant() takes 1 positional argument but 2 were given

Retrieved Context for 'Installation':
{
  "error": "Failed to retrieve context: 500",
  "details": "{\"detail\":\"Failed to retrieve context.\"}"
}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Search Results for 'How to install dependencies':
{
  "error": "Search failed: 500",
  "details": "{\"detail\":\"Search failed.\"}"
}


In [6]:
# Test Code

import os
import time
import json
import requests

# Define the path for the sample README
readme_path = "sample_README.md"

# Sample README content
sample_readme_content = """
# Sample Project

## Introduction

This is a sample README file for testing purposes.

## Installation

Instructions to install...

### Prerequisites

List of prerequisites...

### Step-by-Step Guide

Step-by-step installation guide...

## Usage

How to use the project...
"""

# Write the sample README content to the file
with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(sample_readme_content)

# Wait briefly to ensure the server is up
time.sleep(3)  # Adjust if necessary based on server startup time

# Function to send POST request to /create_memory
def send_create_memory(title, metadata):
    url = "http://localhost:8000/create_memory"
    headers = {"Content-Type": "application/json"}
    body = {
        "content": title,
        "metadata": metadata
    }
    try:
        response = requests.post(url, headers=headers, json=body)
        return response.json()
    except Exception as e:
        return {"error": str(e)}

# Process the README and send sections to the database
from threading import Thread

def run_processing():
    process_readme_and_send(readme_path)

processing_thread = Thread(target=run_processing)
processing_thread.start()

# Wait for processing to complete
processing_thread.join()

# Function to retrieve context for a section
def get_section_context(section_heading, depth=1):
    url = f"http://localhost:8000/context/{section_heading}"
    params = {"depth": depth}
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Failed to retrieve context: {response.status_code}", "details": response.text}
    except Exception as e:
        return {"error": str(e)}

# Retrieve context for the 'Installation' section
context_response = get_section_context("Installation", depth=1)
print("\nRetrieved Context for 'Installation':")
print(json.dumps(context_response, indent=2))

# Function to perform a search
def perform_search(query, top_k=5):
    url = "http://localhost:8000/search"
    headers = {"Content-Type": "application/json"}
    body = {
        "query": query,
        "top_k": top_k
    }
    try:
        response = requests.post(url, headers=headers, json=body)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": f"Search failed: {response.status_code}", "details": response.text}
    except Exception as e:
        return {"error": str(e)}

# Perform a search for "How to install dependencies"
search_query = "How to install dependencies"
search_response = perform_search(search_query, top_k=3)
print(f"\nSearch Results for '{search_query}':")
print(json.dumps(search_response, indent=2))

# Function to prune sections (optional)
def prune_sections(threshold=0.5, max_age_days=30):
    url = "http://localhost:8000/prune"
    headers = {"Content-Type": "application/json"}
    body = {
        "threshold": threshold,
        "max_age_days": max_age_days
    }
    try:
        response = requests.post(url, headers=headers, json=body)
        return response.json()
    except Exception as e:
        return {"error": str(e)}

# Optionally, prune sections
# prune_response = prune_sections()
# print(f"\nPrune Response: {prune_response}")

# Function to rebuild KNN index (optional)
def rebuild_knn():
    url = "http://localhost:8000/rebuild_knn_index"
    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(url, headers=headers)
        return response.json()
    except Exception as e:
        return {"error": str(e)}

# Optionally, rebuild KNN index
# rebuild_response = rebuild_knn()
# print(f"\nRebuild KNN Index Response: {rebuild_response}")

# Clean up: Remove the sample README file
os.remove(readme_path)


ERROR:__main__:Request failed with status code 422: {"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}
ERROR:__main__:Section 'Sample Project' has no vector.


Sent section 'Sample Project' to database. Response: {'error': 'Request failed with status code 422', 'details': '{"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}'}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:__main__:Request failed with status code 422: {"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/advanced_readme_sections/points?wait=true "HTTP/1.1 200 OK"
INFO:__main__:Section 'Introduction' added to Qdrant with ID 9cff1cd4-a453-4578-a769-ba8eec5647b7.


Sent section 'Introduction' to database. Response: {'error': 'Request failed with status code 422', 'details': '{"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}'}


...

Sent section 'Step-by-Step Guide' to database. Response: {'error': 'Request failed with status code 422', 'details': '{"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}'}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:__main__:Request failed with status code 422: {"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/advanced_readme_sections/points?wait=true "HTTP/1.1 200 OK"
INFO:__main__:Section 'Usage' added to Qdrant with ID fb1a01b9-146f-4a2c-a754-6b95fa69b2d7.


Sent section 'Usage' to database. Response: {'error': 'Request failed with status code 422', 'details': '{"detail":[{"type":"missing","loc":["query","content"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.7/v/missing"}]}'}

Retrieved Context for 'Installation':
{
  "error": "Failed to retrieve context: 404",
  "details": "{\"detail\":\"Not Found\"}"
}

Search Results for 'How to install dependencies':
{
  "error": "Search failed: 404",
  "details": "{\"detail\":\"Not Found\"}"
}


: 